In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.utils import shuffle
     
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

2024-08-05 17:04:54.372079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 17:04:54.372219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 17:04:54.490704: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
df_test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')

/tmp/ipykernel_24/2653809078.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')


In [3]:
df.head()

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,55.828924,54.367448,62.025701,3.152561,27.476170,0.00000,3.982447,2.967674,-7.658253,8.360566,5.784302,0.901302,Right Leg - Upper,Right Leg,1.543016,2.628592e-03,7.091360,0.000000,9.307003,0.00000,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,44.064040,48.861520,55.362360,0.919497,12.235290,0.00000,6.347830,1.781713,-6.500838,6.839008,4.987244,0.639885,Head & Neck,Head & Neck,0.821918,1.334303e-07,2.116402,0.000000,3.354148,0.00000,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,Memorial Sloan Kettering Cancer Center,CC-BY,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,62.909730,53.961180,61.670520,3.265153,24.184620,0.00000,5.447655,4.485044,-7.709336,9.092376,6.290359,0.932147,Torso Back Top Third,Torso Back,1.194905,2.959177e-04,4.798335,0.000000,8.886309,0.00000,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,60.060388,18.649518,23.314841,6.079940,14.889242,0.51452,2.077572,0.326788,-4.665323,4.783413,6.400196,0.654458,Torso Front Top Half,Torso Front,2.481328,2.198945e+01,1.975874,1.771705,9.514499,0.66469,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,ACEMID MIA,CC-0,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,52.041180,46.276310,54.855740,2.101708,19.902560,0.00000,4.668053,0.754434,-8.579431,9.148495,6.531302,0.946448,Torso Front Top Half,Torso Front,0.929916,1.378832e-03,3.658854,0.000000,6.467562,0.00000,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


## Constants

In [4]:
RANDOM_STATE = 42
IMAGE_SIZE=(224, 224)
BATCH_SIZE = 32
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


img_augmentation_layers = [
    layers.RandomRotation(factor=0.15),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomFlip(),
    layers.RandomContrast(factor=0.1),
]




## Helper functions

In [5]:
import tensorflow as tf
from typing import NamedTuple, Optional
from sklearn import preprocessing 


def add_path(path):
    return "/kaggle/input/isic-2024-challenge/train-image/image/"+path+'.jpg'

def get_sample():
    df_positive = df[df['target']==1].reset_index(drop=True)
    df_negative = df[df['target']==0].reset_index(drop=True)
    df_negative= resample(df_negative,n_samples=len(df_positive)*20,random_state=RANDOM_STATE)
#     df_positive = resample(df_positive,n_samples=len(df_positive),random_state=RANDOM_STATE)
    
    df_concat = pd.concat([df_positive,df_negative])
    df_concat = shuffle(df_concat).reset_index(drop=True)
    
    print(f'positive shape: {df_positive.shape}, negative shape: {df_negative.shape}')
    print(f'Full dataset shape: {df_concat.shape}')
    
    return df_concat

def visualize(df,idx):
    images = []
    labels = []
    for id in idx:
        current_row = df.iloc[id]
        images.append(plt.imread(current_row["file_path"]))
        labels.append(current_row["target"])
    
    for i, (image, label) in enumerate(zip(images, labels), start=1):
        plt.subplot(3, 3, i)  # Adjusted to start from 1
        plt.imshow(image)
        plt.axis("off")
        plt.title(label)
    plt.show()

def encode(df, columns):
    label_encoder = preprocessing.LabelEncoder() 
    for col in columns:
        df[col]= label_encoder.fit_transform(df[col])
    return df

def scale_columns(df, columns):
    scaler = preprocessing.StandardScaler()
    
    # Transform each specified column
    for col in columns:
        df[col] = scaler.fit_transform(df[[col]])
        
    return df

class Metadata(NamedTuple):
    image: Optional[tf.Tensor] = None
    metadata: Optional[tf.Tensor] = None



def img_augmentation(images):
    for layer in img_augmentation_layers:
        images = layer(images)
    return images

def preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = image / 255.0
    image = img_augmentation(image)
    return image





def preprocess_metadata(filtered_dict):
    # Convert metadata to tensor
    tensor_dict = tf.stack(list(filtered_dict.values()), axis=-1)
    return tensor_dict

def load_image_and_metadata(metadata, image_size):
    image = preprocess_image(metadata['file_path'], image_size)
    filtered_dict = {k: v for k, v in metadata.items() if k != 'file_path'}
    metadata_tensor = preprocess_metadata(filtered_dict)
    return Metadata(image=image, metadata=metadata_tensor)


def unbatch_test_dataset(test_dataset,model):
    unbatched_dataset = test_dataset.unbatch()

    # Initialize lists to store true labels and predictions
    images = []
    labels = []


    # Iterate over the unbatched dataset to get predictions and true labels
    for img_batch, label_batch in test_dataset:
        images.append(img_batch)
        labels.append(label_batch)

    # Convert lists to numpy arrays
    images = np.concatenate(images, axis=0)
    labels = np.concatenate(labels, axis=0)

    # Make predictions on the entire dataset
    y_pred = model.predict(images).ravel()  # Flatten predictions if necessary

    # Convert labels to a flat array
    y_true = labels.ravel()
    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return y_true, y_pred


In [6]:
def calculate_weight(pos,neg):

    # Total number of samples
    total = pos + neg

    # Calculate class weights
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weight = {0: weight_for_0, 1: weight_for_1}

    print('Weight for class 0 (Negative): {:.2f}'.format(weight_for_0))
    print('Weight for class 1 (Positive): {:.2f}'.format(weight_for_1))
    return class_weight

In [7]:
# random_choices = random.choices(list(range(df.shape[0])), k=9)
# visualize(df,random_choices)

In [8]:
def create_dataset(df, image_size=(139, 139), batch_size=32, training=False, return_type='both'):
    labels = df[['target']]

    dataset = tf.data.Dataset.from_tensor_slices((df['file_path'].values, labels))
    dataset = dataset.map(lambda x, y: (preprocess_image(x, image_size), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # Shuffle and batch the dataset
    if training:
        dataset = dataset.shuffle(len(df))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

from sklearn.metrics import roc_auc_score
import numpy as np

def compute_partial_auc(y_true, y_pred, min_tpr):
    """
    Computes the partial AUC score given true labels, predicted scores, and minimum TPR.

    Parameters:
    y_true (np.array): True binary labels.
    y_pred (np.array): Predicted probabilities or scores.
    min_tpr (float): Minimum true positive rate (TPR) to consider for partial AUC.

    Returns:
    partial_auc (float): The computed partial AUC score.
    """
    # Compute the transformed ground truth and predictions
    v_gt = np.abs(y_true - 1)
    v_pred = 1.0 - y_pred

    # Compute the maximum false positive rate (FPR) based on the minimum TPR
    max_fpr = 1 - min_tpr

    # Compute the partial AUC using roc_auc_score
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)

    # Rescale the partial AUC
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) * (partial_auc_scaled - 0.5)

    return partial_auc

# Example usage
# y_true = np.array([...])  # Replace with your true labels
# y_pred = np.array([...])  # Replace with your predicted scores
# min_tpr = 0.9  # Example minimum TPR value
# partial_auc = compute_partial_auc(y_true, y_pred, min_tpr)
# print(f"Partial AUC: {partial_auc:.4f}")


In [9]:
def create_dataset(df, image_size=(139, 139), batch_size=32, training=False, return_type='both'):
    # Extract labels
    labels = df[['target']]
    
    # Depending on the return type, prepare the dataset accordingly
    if return_type == 'both':
        dataset = tf.data.Dataset.from_tensor_slices((dict(df.drop(columns=['target'])), labels))
        dataset = dataset.map(lambda x, y: (load_image_and_metadata(x, image_size), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    elif return_type == 'metadata':
        dataset = tf.data.Dataset.from_tensor_slices((dict(df.drop(columns=['target'])), labels))
        dataset = dataset.map(lambda x, y: (preprocess_metadata({k: v for k, v in x.items() if k != 'file_path'}), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    elif return_type == 'images':
        dataset = tf.data.Dataset.from_tensor_slices((df['file_path'].values, labels))
        dataset = dataset.map(lambda x, y: (preprocess_image(x, image_size), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
    else:
        raise ValueError("Invalid return_type. Must be one of ['both', 'metadata', 'images']")
    
    # Shuffle and batch the dataset
    if training:
        dataset = dataset.shuffle(len(df))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return dataset

from sklearn.metrics import roc_auc_score
import numpy as np

def compute_partial_auc(y_true, y_pred, min_tpr):
    """
    Computes the partial AUC score given true labels, predicted scores, and minimum TPR.

    Parameters:
    y_true (np.array): True binary labels.
    y_pred (np.array): Predicted probabilities or scores.
    min_tpr (float): Minimum true positive rate (TPR) to consider for partial AUC.

    Returns:
    partial_auc (float): The computed partial AUC score.
    """
    # Compute the transformed ground truth and predictions
    v_gt = np.abs(y_true - 1)
    v_pred = 1.0 - y_pred

    # Compute the maximum false positive rate (FPR) based on the minimum TPR
    max_fpr = 1 - min_tpr

    # Compute the partial AUC using roc_auc_score
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)

    # Rescale the partial AUC
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) * (partial_auc_scaled - 0.5)

    return partial_auc

# Example usage
# y_true = np.array([...])  # Replace with your true labels
# y_pred = np.array([...])  # Replace with your predicted scores
# min_tpr = 0.9  # Example minimum TPR value
# partial_auc = compute_partial_auc(y_true, y_pred, min_tpr)
# print(f"Partial AUC: {partial_auc:.4f}")


In [10]:
df_concat = get_sample()

positive shape: (393, 55), negative shape: (7860, 55)
Full dataset shape: (8253, 55)


In [11]:
df_concat['file_path'] = df_concat['isic_id'].apply(add_path)

## Vision Model

In [12]:
positive = df_concat[df_concat['target']==1]

In [13]:
ones = np.ones(positive.shape[0],dtype=np.float32)

In [14]:

augmented_positive = tf.data.Dataset.from_tensor_slices((positive['file_path'].values, positive['target'].values))
augmented_positive = augmented_positive.map(lambda x, y: (preprocess_image(x), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [15]:

augmented_positive1 = tf.data.Dataset.from_tensor_slices((positive['file_path'].values, positive['target'].values))
augmented_positive1 = augmented_positive1.map(lambda x, y: (preprocess_image(x), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [16]:
# df_concat['image'] = df_concat['file_path'].apply(preprocess_image)

dataset = tf.data.Dataset.from_tensor_slices((df_concat['file_path'].values, df_concat['target']))
dataset = dataset.map(lambda x, y: (preprocess_image(x), y),
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [17]:
dataset = dataset.concatenate(augmented_positive,name='first')
dataset = dataset.concatenate(augmented_positive1,name='version2')

In [18]:
dataset = dataset.shuffle(len(dataset))
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [19]:
IMAGE_SIZE[0]

224

In [20]:
efficient = keras.applications.EfficientNetB0(include_top=True, weights=None,input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[0], 3),)

In [21]:
len(efficient.layers)

240

In [22]:
efficient.get_layer(name='block4c_add').output

<KerasTensor shape=(None, 14, 14, 80), dtype=float32, sparse=False, name=keras_tensor_118>

In [23]:
layer_index = None
for i, layer in enumerate(efficient.layers):
    if layer.name == 'block4c_add':
        layer_index = i
        break

In [24]:
efficient.layers[118].name

'block4c_add'

In [25]:
def create_vision_encoder(trainable_layer,trainable=False,input_shape=(139,139,3)):
    
#     resnet_v2 = keras.applications.ResNet50V2(include_top=False, weights="imagenet", pooling="max")
    model = keras.applications.EfficientNetB0(include_top=True, weights='imagenet',input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[0], 3),pooling='avg')
    if trainable:
        for layer in model.layers[:trainable_layer]:
            layer.trainable = trainable
    
    
    
    image = keras.Input(shape=input_shape, name="image")
    
    preprocessed = keras.applications.efficientnet.preprocess_input(image)
    
    
    outputs = model(preprocessed)
#     outputs = model.get_layer(name='block4c_add').output
    outputs = model.layers[118].output
#     outputs = layers.GlobalAveragePooling2D()(outputs)
    return keras.Model([image], outputs, name="vision_encoder")

In [26]:
vision_encoder = create_vision_encoder(trainable_layer=100,input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[0],3),trainable=True)
# vision_encoder.summary()

21834768/21834768 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [27]:
def vision_model():
    vision_encoder = create_vision_encoder(trainable_layer=100,input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[0],3),trainable=True)
#     x = layers.Dense(128,activation='relu')(vision_encoder.output)
#     x = layers.GlobalAveragePooling2D()(vision_encoder.output)
    x = layers.Dense(64,activation='relu')(vision_encoder.output)
    x = layers.Dense(1,activation='sigmoid')(x)

    vision_model = keras.Model(inputs = vision_encoder.input, outputs=x)
    vision_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=METRICS)
    return vision_model

In [28]:
model = vision_model()

In [29]:
# model.summary()

In [30]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [31]:
shuffled_dataset = dataset.shuffle(len(dataset))
# num_test_samples = int(len(dataset) * 0.2)  # Assuming 20% of the dataset for testing
# test_dataset = shuffled_dataset.take(num_test_samples)
# train_dataset = shuffled_dataset.take(int(len(dataset) * 0.8))

In [32]:
# len(test_dataset)

In [33]:
# for image, label in dataset.take(1):
#     print(label)

In [34]:
# model = vision_model()
# history = model.fit(dataset, epochs=1)

In [35]:
class_weight = calculate_weight(7860,11790)
class_weight

Weight for class 0 (Negative): 0.83
Weight for class 1 (Positive): 1.25


{0: 0.8333333333333334, 1: 1.25}

In [36]:
# skf = StratifiedKFold(n_splits=5)
# fold_no = 1
# results = []
# images = df_concat[['file_path']]
# target = df_concat[['target']]

# for train_index, val_index in skf.split(images, target):
#     print(f'Fold {fold_no}...')
    
#     X_train, X_val = images.iloc[train_index], images.iloc[val_index]
#     y_train, y_val = target.iloc[train_index], target.iloc[val_index]
    
#     train_df = pd.concat([X_train, y_train],axis=1)
#     test_df = pd.concat([X_val, y_val],axis=1)
#     print(train_df.shape,test_df.shape)
    
#     vision_train_ds = create_dataset(train_df, IMAGE_SIZE, BATCH_SIZE,training=True,return_type='images')
#     vision_test_ds = create_dataset(test_df, IMAGE_SIZE, BATCH_SIZE,return_type='images')
#     # Instantiate the model for each fold
#     model = vision_model()
    
#     # Train the model
#     history = model.fit(vision_train_ds, epochs=2, validation_data=vision_test_ds,class_weight=class_weight)
    
#     # Evaluate the model on the validation set
#     y_true,y_pred = unbatch_test_dataset(vision_test_ds,model)
#     score = compute_partial_auc(y_true,y_pred, min_tpr=0.80)
# #     scores = model.evaluate(vision_test_ds, verbose=0)
#     print(f'Validation auc: {score}')
    
#     results.append(score)
#     fold_no += 1

# # Calculate the average performance
# # average_score = np.mean(results, axis=0)
# # print(f'\nAverage validation loss: {average_score[0]} - Average validation accuracy: {average_score[1]}')


### Metadata Transformations 

In [37]:
df_concat.columns

Index(['isic_id', 'target', 'patient_id', 'age_approx', 'sex',
       'anatom_site_general', 'clin_size_long_diam_mm', 'image_type',
       'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
       'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
       'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'attribution', 'copyright_license', 'lesion_id',
       'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
       'mel_mitotic_index', '

In [38]:
drop_columns = [
    'isic_id', 'patient_id', 'image_type', 'tbp_tile_type', 'attribution', 
    'copyright_license', 'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 
    'iddx_3', 'iddx_4', 'iddx_5', 'mel_mitotic_index', 'mel_thick_mm', 
    'tbp_lv_dnn_lesion_confidence'
]
df_concat = df_concat.drop(drop_columns,axis=1)

In [39]:
df_concat.shape

(8253, 40)

In [40]:
df_concat.head()

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,file_path
0,0,40.0,female,anterior torso,4.26,20.741267,12.820101,30.964684,24.169809,37.269449,27.359361,56.184392,62.057718,64.060066,78.465136,12.948020,14.417523,2.670875,7.921166,6.794875,-14.405070,16.122935,9.402972,0.515338,Torso Front Bottom Half,Torso Front,3.777694,99.612778,1.362769,7.217264,13.663030,2.154920,5.976117,1.935768,0.138418,115,21.398697,937.951111,-128.577576,/kaggle/input/isic-2024-challenge/train-image/...
1,0,65.0,male,anterior torso,2.72,19.814113,13.344221,30.975995,26.359986,36.771066,29.545171,57.394568,63.150090,55.636695,65.752223,3.002439,15.651030,0.748261,6.469892,4.616008,-10.115529,11.316395,7.560383,0.837024,Torso Front Bottom Half,Torso Front,1.438223,81.883538,2.816692,1.911322,6.855018,0.531313,2.431932,2.913949,0.304348,45,-118.964722,984.857361,-37.767548,/kaggle/input/isic-2024-challenge/train-image/...
2,0,55.0,female,lower extremity,2.88,19.332857,14.045054,30.066025,27.458099,35.745283,30.841704,57.258439,62.909865,38.068834,46.853877,5.179208,14.275423,0.786866,5.287803,2.607926,-8.785042,9.208635,7.971595,0.528121,Right Leg - Upper,Right Leg,2.328767,2.902333,2.276782,2.477788,8.598569,0.864889,1.759677,3.356490,0.265306,0,222.023621,622.243896,98.015228,/kaggle/input/isic-2024-challenge/train-image/...
3,0,80.0,female,posterior torso,3.31,14.487623,11.256026,25.721046,24.745289,29.520559,27.185060,60.609201,65.540375,28.914877,33.987785,5.329330,16.369810,0.505604,3.231597,0.975757,-5.072908,5.322546,5.643407,0.802881,Torso Back Top Third,Torso Back,2.021650,0.967152,2.533615,1.764214,9.340242,0.668361,1.009984,2.328846,0.248447,135,170.637390,1270.703979,71.772034,/kaggle/input/isic-2024-challenge/train-image/...
4,0,65.0,male,posterior torso,3.47,18.681561,16.487633,25.410689,26.319228,31.538926,31.057105,53.677226,57.934988,49.351034,58.280699,5.441922,26.606809,0.679675,2.193928,-0.908539,-8.929665,9.095377,6.615330,0.784241,Torso Back Top Third,Torso Back,2.339219,0.005842,5.663917,1.889498,12.032962,0.582977,2.681651,2.767671,0.419162,85,-71.839172,1516.213745,44.724854,/kaggle/input/isic-2024-challenge/train-image/...


In [41]:
df_concat.dtypes

target                           int64
age_approx                     float64
sex                             object
anatom_site_general             object
clin_size_long_diam_mm         float64
tbp_lv_A                       float64
tbp_lv_Aext                    float64
tbp_lv_B                       float64
tbp_lv_Bext                    float64
tbp_lv_C                       float64
tbp_lv_Cext                    float64
tbp_lv_H                       float64
tbp_lv_Hext                    float64
tbp_lv_L                       float64
tbp_lv_Lext                    float64
tbp_lv_areaMM2                 float64
tbp_lv_area_perim_ratio        float64
tbp_lv_color_std_mean          float64
tbp_lv_deltaA                  float64
tbp_lv_deltaB                  float64
tbp_lv_deltaL                  float64
tbp_lv_deltaLB                 float64
tbp_lv_deltaLBnorm             float64
tbp_lv_eccentricity            float64
tbp_lv_location                 object
tbp_lv_location_simple   

In [42]:
df_concat  = encode(df_concat,['sex','anatom_site_general','tbp_lv_location','tbp_lv_location_simple'])

In [43]:
df_concat.head()

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,file_path
0,0,40.0,0,0,4.26,20.741267,12.820101,30.964684,24.169809,37.269449,27.359361,56.184392,62.057718,64.060066,78.465136,12.948020,14.417523,2.670875,7.921166,6.794875,-14.405070,16.122935,9.402972,0.515338,17,6,3.777694,99.612778,1.362769,7.217264,13.663030,2.154920,5.976117,1.935768,0.138418,115,21.398697,937.951111,-128.577576,/kaggle/input/isic-2024-challenge/train-image/...
1,0,65.0,1,0,2.72,19.814113,13.344221,30.975995,26.359986,36.771066,29.545171,57.394568,63.150090,55.636695,65.752223,3.002439,15.651030,0.748261,6.469892,4.616008,-10.115529,11.316395,7.560383,0.837024,17,6,1.438223,81.883538,2.816692,1.911322,6.855018,0.531313,2.431932,2.913949,0.304348,45,-118.964722,984.857361,-37.767548,/kaggle/input/isic-2024-challenge/train-image/...
2,0,55.0,0,2,2.88,19.332857,14.045054,30.066025,27.458099,35.745283,30.841704,57.258439,62.909865,38.068834,46.853877,5.179208,14.275423,0.786866,5.287803,2.607926,-8.785042,9.208635,7.971595,0.528121,12,4,2.328767,2.902333,2.276782,2.477788,8.598569,0.864889,1.759677,3.356490,0.265306,0,222.023621,622.243896,98.015228,/kaggle/input/isic-2024-challenge/train-image/...
3,0,80.0,0,3,3.31,14.487623,11.256026,25.721046,24.745289,29.520559,27.185060,60.609201,65.540375,28.914877,33.987785,5.329330,16.369810,0.505604,3.231597,0.975757,-5.072908,5.322546,5.643407,0.802881,15,5,2.021650,0.967152,2.533615,1.764214,9.340242,0.668361,1.009984,2.328846,0.248447,135,170.637390,1270.703979,71.772034,/kaggle/input/isic-2024-challenge/train-image/...
4,0,65.0,1,3,3.47,18.681561,16.487633,25.410689,26.319228,31.538926,31.057105,53.677226,57.934988,49.351034,58.280699,5.441922,26.606809,0.679675,2.193928,-0.908539,-8.929665,9.095377,6.615330,0.784241,15,5,2.339219,0.005842,5.663917,1.889498,12.032962,0.582977,2.681651,2.767671,0.419162,85,-71.839172,1516.213745,44.724854,/kaggle/input/isic-2024-challenge/train-image/...


In [44]:
df_concat.columns

Index(['target', 'age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B',
       'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext',
       'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', 'file_path'],
      dtype='object')

In [45]:
columns = ['age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B',
       'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext',
       'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z']

In [46]:
df_concat = scale_columns(df_concat,columns)

In [47]:
df_concat.head()

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,file_path
0,0,-1.322097,-1.406725,-1.584460,0.130828,0.173879,-0.592195,0.535009,-0.582471,0.447070,-0.721556,0.306844,0.213452,2.000791,2.262959,0.333670,-0.870041,1.906491,1.110468,2.403406,-1.532695,1.855043,0.707411,-1.584354,0.995758,1.100169,0.903157,1.474368,-1.216956,1.884197,0.225826,1.414183,1.797178,-0.482701,-1.347099,0.535118,0.153452,-0.239167,-2.082825,/kaggle/input/isic-2024-challenge/train-image/...
1,0,0.515668,0.557359,-1.584460,-0.667628,-0.049881,-0.449380,0.537114,-0.100060,0.361253,-0.274428,0.515660,0.398154,1.228358,1.207092,-0.542200,-0.641410,-0.431130,0.553553,1.454433,-0.321466,0.503787,-0.015436,0.678748,0.995758,1.100169,-0.889478,1.047513,-0.373455,-0.583453,-0.802694,-0.653930,-0.181373,0.975404,-0.023783,-0.791193,-0.565957,-0.127658,-1.057131,/kaggle/input/isic-2024-challenge/train-image/...
2,0,-0.219438,-1.406725,-0.154469,-0.584672,-0.166027,-0.258414,0.367709,0.141812,0.184621,-0.009210,0.492171,0.357536,-0.382636,-0.362506,-0.350500,-0.896379,-0.384192,0.099935,0.579843,0.054220,-0.088764,0.145883,-1.494424,0.084794,0.004514,-0.207093,-0.854063,-0.686687,-0.320005,-0.539287,-0.229028,-0.556662,1.635069,-0.335146,-1.643822,1.181722,-0.989689,0.476528,/kaggle/input/isic-2024-challenge/train-image/...
3,0,1.618327,-1.406725,0.560526,-0.361726,-1.335379,-1.018382,-0.441178,-0.455715,-0.887227,-0.757211,1.070345,0.802311,-1.222065,-1.431095,-0.337279,-0.508184,-0.726165,-0.689119,-0.131024,1.102408,-1.181255,-0.767464,0.438545,0.631372,0.552342,-0.442423,-0.900655,-0.537684,-0.651869,-0.427239,-0.479361,-0.975180,0.103233,-0.469599,0.914065,0.918351,0.551878,0.180112,/kaggle/input/isic-2024-challenge/train-image/...
4,0,0.515668,0.557359,0.560526,-0.278770,-0.323211,0.407153,-0.498956,-0.109037,-0.539681,0.034853,-0.125767,-0.483634,0.651955,0.586546,-0.327364,1.389247,-0.514519,-1.087316,-0.951701,0.013383,-0.120605,-0.386179,0.307409,0.631372,0.552342,-0.199084,-0.923800,1.278377,-0.593603,-0.020437,-0.588122,-0.041967,0.757359,0.891876,-0.033301,-0.324423,1.135522,-0.125384,/kaggle/input/isic-2024-challenge/train-image/...


In [48]:
# train_df, test_df = train_test_split(df_concat, test_size=0.11, 
#                                      stratify=df_concat["target"].values, random_state=42)
# # 5% for validation
# # train_df, val_df = train_test_split(train_df, test_size=0.11,stratify=train_df["target"].values, random_state=42)

# print(f"Total training examples: {len(train_df)}")
# # print(f"Total validation examples: {len(val_df)}")
# print(f"Total test examples: {len(test_df)}")

In [49]:
# labels = df['target']

In [50]:
# labels.shape

https://github.com/sayakpaul/Multimodal-Entailment-Baseline/blob/main/multimodal_entailment.ipynb

In [51]:
class ProjectEmbeddings(keras.layers.Layer):
    def __init__(self, num_projection_layers, projection_dims, dropout_rate, **kwargs):
        super(ProjectEmbeddings, self).__init__(**kwargs)
        self.num_projection_layers = num_projection_layers
        self.projection_dims = projection_dims
        self.dropout_rate = dropout_rate
        
        # Initialize Dense and Dropout layers in __init__ or build
        self.projection_layers = [layers.Dense(self.projection_dims) for _ in range(self.num_projection_layers)]
        self.dropout_layers = [layers.Dropout(self.dropout_rate) for _ in range(self.num_projection_layers - 1)]

    def call(self, embeddings):
        x = embeddings
        for i in range(self.num_projection_layers):
            x = self.projection_layers[i](x)
            if i < self.num_projection_layers - 1:
                x = tf.nn.gelu(x)
                x = self.dropout_layers[i](x)
        return x

In [52]:
df_concat.shape

(8253, 40)

(7860, 56), negative shape: (11790, 56)

In [53]:
# vision_test_ds

In [54]:
# # Create the dataset
# vision_train_ds = create_dataset(train_df, IMAGE_SIZE, batch_size,training=True,return_type='images')
# vision_test_ds = create_dataset(test_df, IMAGE_SIZE, batch_size,return_type='images')
# # val_ds = create_dataset(val_df, image_size, batch_size)

In [55]:
# vision_history = vision_model.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=20,
#     class_weight=class_weight,
#     callbacks=[partial_auc_callback]
# )

In [56]:
# for (images, metadata), labels in train_ds.take(1):
#     print(images.shape,metadata.shape,labels.shape)

In [57]:
# !pip install -q focal-loss

In [58]:
# from focal_loss import BinaryFocalLoss

In [59]:
# def create_table_encoder(
#     num_projection_layers, projection_dims, dropout_rate, trainable=False
# ):
#     table_input = tf.keras.Input(shape=(38,))
#     dense_1024 = tf.keras.layers.Dense(1024, activation='relu')(table_input)
#     dense_512 = tf.keras.layers.Dense(512, activation='relu')(dense_1024)

#     # Project the embeddings produced by the model.
#     outputs = ProjectEmbeddings(num_projection_layers, projection_dims, dropout_rate)(dense_512)

#     # Create the text encoder model.
#     return keras.Model(table_input, outputs, name="table_encoder")

In [60]:
# table_encoder = create_table_encoder(num_projection_layers=1, projection_dims=256, dropout_rate=0.1)
# # keras.utils.plot_model(table_encoder, show_shapes=True)

In [61]:
# def create_multimodal_model(num_projection_layers=1, projection_dims=256, dropout_rate=0.1, 
#                      vision_trainable=False, text_trainable=False):
#     # Receive the images as inputs.
#     image_1 = keras.Input(shape=(139, 139, 3), name="image_1")
#     table_input = tf.keras.Input(shape=(38,))

    

#     # Create the encoders.
#     vision_encoder = create_vision_encoder(num_projection_layers, projection_dims, dropout_rate, vision_trainable)
#     table_encoder = create_table_encoder(num_projection_layers, projection_dims, dropout_rate, text_trainable)

#     # Fetch the embedding projections.
#     vision_projections = vision_encoder(image_1)
#     table_projections = table_encoder(table_input)

#     # Concatenate the projections and pass through the classification layer.
#     concatenated = keras.layers.Concatenate()([vision_projections, table_projections])
#     outputs = keras.layers.Dense(1, activation="sigmoid")(concatenated)
#     return keras.Model([image_1, table_input], outputs)

In [62]:
# vision_model = create_vision_encoder(2, 256, 0.5,True)
# keras.utils.plot_model(vision_model, show_shapes=True)

In [63]:
# vision_model.output

In [64]:

# multimodal_model = create_multimodal_model(vision_trainable=True, text_trainable=True)
# keras.utils.plot_model(multimodal_model, show_shapes=True)

In [65]:
# multimodal_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=METRICS)


In [66]:
# history = multimodal_model.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=10,
#     class_weight=class_weight
# )

In [67]:
# from focal_loss import BinaryFocalLoss

In [68]:
# multimodal_model_focal = create_multimodal_model()
# multimodal_model_focal.compile(
#     optimizer="adam", loss=BinaryFocalLoss(gamma=2), metrics=METRICS
# )
# history = multimodal_model_focal.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=10
# )